In [63]:
import pandas as pd 
import json
from arcgis.features import FeatureLayer

In [64]:
# url = 'https://services1.arcgis.com/79kfd2K6fskCAkyg/arcgis/rest/services/FoodServiceData/FeatureServer/0'

# batch_size = 1000  # Number of records to retrieve per batch
# offset = 0  # Initial offset value
# data_list = []
# # ​Create the feature layer object
# feature_layer = FeatureLayer(url)

# while True:
#     # Query the feature layer with pagination
#     query_result = feature_layer.query(where='1=1', out_fields='*', return_geometry=False, result_offset=offset, result_record_count=batch_size)
    
#     # Retrieve the features from the query result
#     features = query_result.features
    
#     # Process the data for the current batch
#     for feature in features:
#         data_list.append(feature.attributes)
    
#     # Break the loop if the response is empty or the desired number of records is reached
#     if len(features) == 0 or len(data_list) >= 1000:
#         break
    
#     # Increment the offset by the batch size
#     offset += batch_size
# # Create a DataFrame from the data list
# df = pd.DataFrame(data_list)

In [65]:
# df = pd.read_csv("local_working.csv")

In [66]:
cols_drop = ['InspectionID', 'PlaceName', 'Address2', 'TypeDescription', 'NameSearch', 'Intersection']
df.drop(cols_drop, axis=1, inplace=True)

In [67]:
df['InspectionDate'] = pd.to_datetime(df['InspectionDate'])

In [68]:
df.head()

,EstablishmentID,Ins_TypeDesc,EstablishmentName,Address,City,State,Zip,InspectionDate,score,Grade,ObjectId
0,29956,REGULAR,CVS #6211,3721 LEXINGTON RD,LOUISVILLE,KY,40207,2022-07-06,100,None,1
1,29967,REGULAR,NEIGHBORHOOD FOOD MART,542 LAMPTON ST,LOUISVILLE,KY,40203,2022-09-07,97,None,2
2,29979,REGULAR,CIRCLE K #4700075,1630 ARTHUR ST,LOUISVILLE,KY,40208,2022-09-01,97,A,3
3,29979,REGULAR,CIRCLE K #4700075,1630 ARTHUR ST,LOUISVILLE,KY,40208,2022-09-01,98,None,4
4,29979,REGULAR,CIRCLE K #4700075,1630 ARTHUR ST,LOUISVILLE,KY,40208,2023-01-27,95,A,5


In [69]:
# # Grouping by EstablishmentID and aggregating the scores and inspection dates
# grouped_df = df.groupby("EstablishmentID").agg({
#     "EstablishmentName": "first",
#     "score": lambda x: list(x),
#     "InspectionDate": lambda x: list(x)
# })

# # Sorting the establishments based on the count of low scores
# grouped_df["low_score_count"] = grouped_df["score"].apply(lambda x: sum(score < 80 for score in x))
# grouped_df = grouped_df.sort_values("low_score_count", ascending=False)

# # Selecting the top 20 establishments with the most low scores
# top_20_establishments = grouped_df.head(20)

# # Printing the results
# top_20_establishments[["EstablishmentName", "score", "InspectionDate"]]

In [74]:
# Grouping by EstablishmentID and aggregating the necessary columns
grouped_df = df.groupby("EstablishmentID").agg({
    "EstablishmentName": "first",
    "Address": "first",
    "score": lambda x: list(x)
})

# Sorting the establishments based on the count of low scores
grouped_df["low_score_count"] = grouped_df["score"].apply(lambda x: sum(score < 75 for score in x))
grouped_df = grouped_df.sort_values("low_score_count", ascending=False)

# Selecting the top 20 establishments with the most low scores
top_20_establishments = grouped_df.head(20)

# Creating the final DataFrame with the desired columns
final_df = top_20_establishments[["EstablishmentName", "Address", "score"]]

# Printing the final DataFrame
final_df.head(30)

,EstablishmentName,Address,score
EstablishmentID,,,
115438,EXPRESSWAY FOOD MART,1420 W BROADWAY,"[0, 93, 96, 73, 91, 0, 78, 93, 0, 98, 0, 0, 0,..."
116093,3 BROTHERS FOOD MARKET,4132 W BROADWAY,"[0, 100, 100, 0, 0, 0, 0, 0, 0, 0, 82, 90, 0, ..."
102268,FAMILY DOLLAR #31859,2421 W MARKET ST,"[90, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
107692,FAMILY TIES,1030 CECIL AVE,"[0, 0, 81, 0, 0, 0, 0, 0, 0, 0, 0, 0, 78, 100]"
107689,BEST WESTERN,4110 DIXIE HWY,"[74, 93, 97, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
117851,MOE'S #2,1636 W OAK ST,"[99, 97, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
118623,CEDAR BAR,416 S 26TH ST,"[91, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
116570,COLE'S EVENT SPACE,2928 W KENTUCKY ST,"[96, 0, 0, 0, 98, 0, 0, 0, 0, 0]"
107585,WENDYS,3610 BUECHEL BYPASS,"[100, 0, 0, 81, 93, 87, 0, 0, 0, 0, 98, 0, 0]"


In [73]:
# Grouping by EstablishmentID and calculating average scores
avg_scores_df = df.groupby("EstablishmentID").agg({
    "EstablishmentName": "first",
    "Address": "first",
    "score": "mean"
})

# Sorting the establishments based on average scores
sorted_restaurants = avg_scores_df.sort_values("score", ascending=True)

# Displaying the restaurants with consistently lower scores
sorted_restaurants.head(40)



,EstablishmentName,Address,score
EstablishmentID,,,
114313,PEACE OF MIND PRESCHOOL,2119 OLD SHEPHERDSVILLE RD,0.000000
108255,MARINATIONS CATERING,2806 DUMESNIL STREET,0.000000
114588,BLACK MARKET,2313 W MARKET ST,0.000000
114608,CEMITAS POBLANAS AZTECA,None,0.000000
97399,SOUL FOOD KITCHEN ON WHEELS,None,0.000000
118623,CEDAR BAR,416 S 26TH ST,8.272727
67696,HOWARD JOHNSON,709 PHILLIPS LN,11.111111
102268,FAMILY DOLLAR #31859,2421 W MARKET ST,14.615385
117851,MOE'S #2,1636 W OAK ST,16.333333
